In [189]:
import pandas as pd
import numpy 
import sys 
import nmslib 
import time 
import numpy as np

In [5]:
!pip install -U pyarrow

     |████████████████████████████████| 29.2MB 2.1MB/s eta 0:00:011
You should consider upgrading via the 'pip install --upgrade pip' command.


In [190]:
# load  userFactors
user1 = pd.read_parquet("userFactors/part-00000-3debbcdc-7517-4f27-8de2-90709fac540b-c000.snappy.parquet",engine='pyarrow')
user2 = pd.read_parquet('userFactors/part-00001-3debbcdc-7517-4f27-8de2-90709fac540b-c000.snappy.parquet',engine='pyarrow')
user3 = pd.read_parquet('userFactors/part-00002-3debbcdc-7517-4f27-8de2-90709fac540b-c000.snappy.parquet',engine='pyarrow')
user4 = pd.read_parquet('userFactors/part-00003-3debbcdc-7517-4f27-8de2-90709fac540b-c000.snappy.parquet',engine='pyarrow')
user5 = pd.read_parquet('userFactors/part-00004-3debbcdc-7517-4f27-8de2-90709fac540b-c000.snappy.parquet',engine='pyarrow')
user6 = pd.read_parquet('userFactors/part-00005-3debbcdc-7517-4f27-8de2-90709fac540b-c000.snappy.parquet',engine='pyarrow')
user7 = pd.read_parquet('userFactors/part-00006-3debbcdc-7517-4f27-8de2-90709fac540b-c000.snappy.parquet',engine='pyarrow')
user8 = pd.read_parquet('userFactors/part-00007-3debbcdc-7517-4f27-8de2-90709fac540b-c000.snappy.parquet',engine='pyarrow')
user9 = pd.read_parquet('userFactors/part-00008-3debbcdc-7517-4f27-8de2-90709fac540b-c000.snappy.parquet',engine='pyarrow')
user10 = pd.read_parquet('userFactors/part-00009-3debbcdc-7517-4f27-8de2-90709fac540b-c000.snappy.parquet',engine='pyarrow')

In [191]:
users = pd.concat([user1,user2,user3,user4,user5,user6,user7,user8,user9,user10],ignore_index=True)
users.head()

,id,features
0,0,"[0.0014444863, 0.008087372, 0.00830631, -0.004..."
1,10,"[3.4017863, 0.9976335, 1.5587792, -1.5164244, ..."
2,20,"[0.15045026, 0.07838725, 0.105005324, -0.25015..."
3,30,"[1.7004522, 0.1540619, 0.4750695, -1.3746185, ..."
4,40,"[0.39282304, 0.018166667, 0.3246519, -1.225637..."


In [192]:
users.shape

(526137, 2)

In [197]:
users.features[0].shape

(10,)

In [198]:
# load itemFactors
item1 = pd.read_parquet("itemFactors/part-00000-67122015-d259-44a8-8f01-bc9c0224b270-c000.snappy.parquet",engine='pyarrow')
item2 = pd.read_parquet('itemFactors/part-00001-67122015-d259-44a8-8f01-bc9c0224b270-c000.snappy.parquet',engine='pyarrow')
item3 = pd.read_parquet('itemFactors/part-00002-67122015-d259-44a8-8f01-bc9c0224b270-c000.snappy.parquet',engine='pyarrow')
item4 = pd.read_parquet('itemFactors/part-00003-67122015-d259-44a8-8f01-bc9c0224b270-c000.snappy.parquet',engine='pyarrow')
item5 = pd.read_parquet('itemFactors/part-00004-67122015-d259-44a8-8f01-bc9c0224b270-c000.snappy.parquet',engine='pyarrow')
item6 = pd.read_parquet('itemFactors/part-00005-67122015-d259-44a8-8f01-bc9c0224b270-c000.snappy.parquet',engine='pyarrow')
item7 = pd.read_parquet('itemFactors/part-00006-67122015-d259-44a8-8f01-bc9c0224b270-c000.snappy.parquet',engine='pyarrow')
item8 = pd.read_parquet('itemFactors/part-00007-67122015-d259-44a8-8f01-bc9c0224b270-c000.snappy.parquet',engine='pyarrow')
item9 = pd.read_parquet('itemFactors/part-00008-67122015-d259-44a8-8f01-bc9c0224b270-c000.snappy.parquet',engine='pyarrow')
item10 = pd.read_parquet('itemFactors/part-00009-67122015-d259-44a8-8f01-bc9c0224b270-c000.snappy.parquet',engine='pyarrow')

In [199]:
items = pd.concat([item1,item2,item3,item4,item5,item6,item7,item8,item9,item10],ignore_index=True)
items.head()

,id,features
0,0,"[0.03876327, 0.10941172, 0.0064438437, -0.0896..."
1,10,"[-0.033253636, -0.022365738, -0.013063856, -0...."
2,20,"[0.12585682, -0.0031638069, 0.07332305, -0.002..."
3,30,"[0.058448564, 0.02108172, 0.02516022, -0.04953..."
4,40,"[0.02590116, -0.005541374, 0.0052622654, -0.00..."


In [200]:
items.shape

(152005, 2)

In [201]:
items.features[0].shape

(10,)

In [202]:
userfactors = np.array(users.features.tolist()).reshape(526137,10)
itemfactors = np.array(items.features.tolist()).reshape(152005,10)

In [203]:
np.sort(userfactors[0].dot(itemfactors.transpose()))[::-1]

array([ 0.00236463,  0.00218299,  0.00217143, ..., -0.00049464,
       -0.00049724, -0.00051629], dtype=float32)

In [204]:
start = time.time()
for i in range(userfactors.shape[0]):
    user_i = userfactors[i]
    score = np.sort(user_i.dot(itemfactors.transpose()))[::-1]
end = time.time()
print('brute-force query time = %f' % (end-start))

brute-force query time = 5826.075924


nmslib

In [223]:
# Set index parameters
M = 20
efC = 300
K =500
num_threads = 4
index_time_params = {'M': M, 'indexThreadQty': num_threads, 'efConstruction': efC, 'post' : 0}

In [224]:
# Intitialize the library, specify the space, the type of the vector and add data points 
index = nmslib.init(method='hnsw', space='cosinesimil') 
index.addDataPointBatch(itemfactors)

152005

In [225]:
# Create an index
start = time.time()
index.createIndex(index_time_params) 
end = time.time() 
print('Index-time parameters', index_time_params)
print('Indexing time = %f' % (end-start))

Index-time parameters {'M': 20, 'indexThreadQty': 4, 'efConstruction': 300, 'post': 0}
Indexing time = 64.634902


In [226]:
# Setting query-time parameters
efS = 100
query_time_params = {'efSearch': efS}
print('Setting query-time parameters', query_time_params)
index.setQueryTimeParams(query_time_params)

Setting query-time parameters {'efSearch': 100}


In [227]:
# Querying
query_qty = userfactors.shape[0]
start = time.time() 
nbrs = index.knnQueryBatch(userfactors, k = K, num_threads = num_threads)
end = time.time() 
print('kNN time total=%f (sec), per query=%f (sec), per query adjusted for thread number=%f (sec)' % 
      (end-start, float(end-start)/query_qty, num_threads*float(end-start)/query_qty))

kNN time total=69.510176 (sec), per query=0.000132 (sec), per query adjusted for thread number=0.000528 (sec)


In [229]:
# Computing gold-standard data 
print('Computing gold-standard data')

start = time.time()
sindx = NearestNeighbors(n_neighbors=K, metric='cosine', algorithm='brute').fit(itemfactors)
end = time.time()

print('Brute-force preparation time %f' % (end - start))

start = time.time() 
gs = sindx.kneighbors(userfactors)
end = time.time()

print('brute-force kNN time total=%f (sec), per query=%f (sec)' % 
      (end-start, float(end-start)/query_qty) )

Computing gold-standard data
Brute-force preparation time 0.001072
brute-force kNN time total=0.001072 (sec), per query=0.000000 (sec)


In [ ]:
# Finally computing recall
recall=0.0
for i in range(0, query_qty):
    correct_set = set(gs[1][i])
    ret_set = set(nbrs[i][0])
    recall = recall + float(len(correct_set.intersection(ret_set))) / len(correct_set)
recall = recall / query_qty
print('kNN recall %f' % recall)